In [1]:
from pandas import read_csv
from torch.optim import SGD
from torch.nn import MSELoss
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter
from torch import randperm
from config import DATA_DIR
from src.data_set import RatingsDataset
from src.model import MF
from src.runner import Runner
from src.utils import (
    create_dataset,
    mine_outliers_scipy,
    mine_outliers_sklearn,
    mine_outliers_torch,
    DataConverter,
    DataProcessor,
    mean_centralised
)
from src.consistency import direct_calculation

"""
The Deam dataset is based on Arousal-Valence 2D emotional model.
The Valence/Arousal ratings were collected using Amazon Mechanical Turks service.
Each turk from the collected crowd were asked to mark his own emotion for the current song on a 2D plane, Arousal/Valence.
For more information please read: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0173392
"""

DF_PATH = f"{DATA_DIR}" \
          f"/DEAM/annotations/annotations per each rater/" \
          f"song_level/static_annotations_songs_1_2000_mean_centralised.csv"

def select_n_random(trainset: RatingsDataset):
    """
    Selects n random data points and their corresponding labels from a dataset
    """
    perm = randperm(len(trainset))
    return trainset[perm][:100]

In [2]:
"""
This block analyze mean-centralized data consistency using the direct calculation defined by:
consistency += row.rating - row.song.mean() for all rows in dataset
"""
columns = ["user_id", "item_id", "rating"]
original_df = read_csv(DF_PATH, skipinitialspace=True, usecols=columns)
original_df.columns = ["user_id", "item_id", "rating"]
mean_centralised(dataframe=original_df)
consistency = direct_calculation(data_frame=original_df)
print(f"Raw data consistency according to direct calculation is: \x1b[31m{consistency}\x1b[0m")

  0%|          | 0/3 [00:00<?, ?it/s]

KeyError: 'rating'

In [2]:
"""
This block of code calculates the outliers alongside the valence axis
"""
columns = ["workerID", "SongId", "Valence"]
original_df = read_csv(DF_PATH, skipinitialspace=True, usecols=columns)
original_df.columns = ["user_id", "item_id", "rating"]
original_df = mean_centralised(dataframe=original_df)

data_converter = DataConverter(
        original_df=original_df, n_random_users=10, n_ratings_per_random_user=9
)

valence_model = MF(
    n_users=data_converter.n_users,
    n_items=data_converter.n_item,
)
epochs = 100

criterion = MSELoss()
optimizer = SGD(valence_model.parameters(), lr=5, weight_decay=1e-3)
runner = Runner(
    model=valence_model,
    criterion=criterion,
    optimizer=optimizer,
    epochs=epochs
)

train_set = create_dataset(data_converter=data_converter)
train_load = DataLoader(train_set, batch_size=1000, shuffle=True)
users, items, ratings = select_n_random(train_set)

with SummaryWriter("runs/DEAM/valence") as writer:
    writer.add_graph(valence_model, (users, items))

    for epoch in range(epochs):
        epoch_loss = runner.train(train_loader=train_load, epoch=epoch, writer=writer)
        print(f"epoch={epoch + 1}, loss={epoch_loss}")

100%|██████████| 18/18 [00:00<00:00, 82.71batch/s, train_loss=0.00541]


epoch=1, loss=0.06094814507797737


100%|██████████| 18/18 [00:00<00:00, 100.18batch/s, train_loss=0.00597]


epoch=2, loss=0.06078394703486335


100%|██████████| 18/18 [00:00<00:00, 87.67batch/s, train_loss=0.0059] 


...

epoch=98, loss=0.008457367343485142


100%|██████████| 18/18 [00:00<00:00, 91.54batch/s, train_loss=0.00087] 


epoch=99, loss=0.008397215937126414


100%|██████████| 18/18 [00:00<00:00, 83.87batch/s, train_loss=0.000754]

epoch=100, loss=0.008303904206636578


In [3]:
"""
This block of code calculates the outliers alongside the arousal axis
"""
columns = ["workerID", "SongId", "Arousal"]
original_df = read_csv(DF_PATH, skipinitialspace=True, usecols=columns)
original_df.columns = ["user_id", "item_id", "rating"]
original_df = mean_centralised(dataframe=original_df)

data_converter = DataConverter(
        original_df=original_df, n_random_users=10, n_ratings_per_random_user=9
)

arousal_model = MF(
    n_users=data_converter.n_users,
    n_items=data_converter.n_item,
)
epochs = 100

criterion = MSELoss()
optimizer = SGD(arousal_model.parameters(), lr=5, weight_decay=1e-3)
runner = Runner(
    model=arousal_model,
    criterion=criterion,
    optimizer=optimizer,
    epochs=epochs,
)

train_set = create_dataset(data_converter=data_converter)
train_load = DataLoader(train_set, batch_size=1000, shuffle=True)
users, items, ratings = select_n_random(train_set)

with SummaryWriter("runs/DEAM/arousal") as writer:
    writer.add_graph(arousal_model, (users, items))

    for epoch in range(epochs):
        epoch_loss = runner.train(train_loader=train_load, epoch=epoch, writer=writer)
        print(f"epoch={epoch + 1}, loss={epoch_loss}")

100%|██████████| 18/18 [00:00<00:00, 88.02batch/s, train_loss=0.00531]


epoch=1, loss=0.06177968693468115


100%|██████████| 18/18 [00:00<00:00, 99.79batch/s, train_loss=0.00548]


epoch=2, loss=0.061219564320808724


100%|██████████| 18/18 [00:00<00:00, 76.59batch/s, train_loss=0.00539]


...

epoch=98, loss=0.008367749842172926


100%|██████████| 18/18 [00:00<00:00, 75.66batch/s, train_loss=0.000901]


epoch=99, loss=0.008399177120480727


100%|██████████| 18/18 [00:00<00:00, 94.28batch/s, train_loss=0.00084] 

epoch=100, loss=0.008326734231589934


In [4]:
valence_outliers = mine_outliers_scipy(model=valence_model, data_converter=data_converter)
arousal_outliers = mine_outliers_scipy(model=arousal_model, data_converter=data_converter)

items_group_by_users = data_converter.original_df.groupby("user_id")
combined_outliers = {}
for user_id, valence_dist in valence_outliers.items():
    arousal_dist = arousal_outliers[user_id]
    combined_outliers[user_id] = valence_dist + arousal_dist

combined_outliers = dict(sorted(combined_outliers.items(), key=lambda item: item[1]))
for user_id, item_id in combined_outliers.items():
    number_of_items = len(items_group_by_users.get_group(user_id))
    print(f"user: {user_id}, dist: {item_id}, #items: {number_of_items}")

user: random_guy_187, dist: -79.87926167514736, #items: 9
user: random_guy_196, dist: -61.504188142567614, #items: 9
user: random_guy_194, dist: -49.64625212429112, #items: 9
user: random_guy_188, dist: -38.71032156074537, #items: 9
user: 19fee46f2810f34a8b69a7768d897a59, dist: -36.5754049091067, #items: 1
user: random_guy_190, dist: -34.09597821371794, #items: 9
user: ff18a27328ffd40ef52b7ebb7a0ded94, dist: -17.543451446355363, #items: 20
user: random_guy_191, dist: -17.20264469597077, #items: 9
user: random_guy_189, dist: -4.519798138322379, #items: 9
user: random_guy_192, dist: -3.8970388278711994, #items: 9
user: fd5b08ce362d855ca9152a894348130c, dist: -2.962559945499283, #items: 222
user: e105c200f413d7b2c5850c0df4b9687e, dist: -1.7301799548594072, #items: 2
user: c3c21239b85dcdd6679fc212afd02a49, dist: 6.671131268637211, #items: 9
user: 7cecbffe1da5ae974952db6c13695afe, dist: 11.214156184287361, #items: 428
user: a0f5cedc3a2371ec13663226c4b44771, dist: 13.959845416054605, #items: